In [98]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

import augmentation
import loss
import dotenv
import torch
import matplotlib.pyplot as plt
import mlflow
import os
from torch.utils.data import DataLoader
from intra import IntrA
from models.pointnet import PointNetCls
from models.pointnetcls import PointNet
from models.pointnet2 import PointNet2
from models.pointconv import PointConvDensityClsSsg
from models.dgcnn import DGCNN
from utils import train_model, train_kfold_intra, eval_model_classification


os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

dotenv.load_dotenv()  # load the MLflow http authentication parameters
mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))

dev = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(dev)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mps


In [108]:
"""Example training setup for single epoch PointNet."""
if True:
    # train pointnet for a single epoch to test utils on a slow laptop
    dataset = IntrA(
        "./data",
        npoints=1024,
        exclude_seg=True,
        norm=True,
        dataset="classification",
    )

    trn, tst = torch.utils.data.random_split(dataset, [0.8, 0.2], torch.Generator().manual_seed(42))
 
    model = PointNet(k=2)

    train_dl = DataLoader(trn, batch_size=8, num_workers=0, drop_last=True, shuffle=True)
    test_dl = DataLoader(tst, batch_size=8, num_workers=0, drop_last=False)

    train_model(
        model,
        train_dl,
        test_dl,
        loss.trans_feat_loss(),
        augmentation.aug_dgcnn,
        epochs=1,
        checkpoint_epoch=50,
        model_name="PointNet",
        trans_loss=True,
        opt="adam",
        sched="step"
    )

In [109]:
"""DGCNN full training setup."""
if True:
    # train pointnet for a single epoch to test utils on a slow laptop
    dataset = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=True,
        dataset="classification",
    )

    trn, tst = torch.utils.data.random_split(dataset, [0.8, 0.2], torch.Generator().manual_seed(42))
 
    model = DGCNN(output_channels=2, norm=True)

    train_dl = DataLoader(trn, batch_size=32, num_workers=8, drop_last=True, shuffle=True)
    test_dl = DataLoader(tst, batch_size=16, num_workers=4, drop_last=False)

    train_model(
        model,
        train_dl,
        test_dl,
        loss.smooth_cross_entropy_loss(),
        augmentation.aug_dgcnn,
        model_name="DGCNN",
        epochs=200,
        checkpoint_epoch=50,
        trans_loss=False,
        opt="sgd",
        sched="cosine",
        lr=0.001
    )

FileNotFoundError: [Errno 2] No such file or directory: '/data/generated/vessel/ad'

In [73]:
train_kfold_intra(
    DGCNN,
    {"output_channels":2, "norm":True},
    epochs=200,
    model_name="DGCNN",
    batch_size=32,
    num_workers=8,
    intra_root="/data",
    npoints=1024,
    norm=True,
    splits="./file_splits/cls/",
    exclude_seg=True,
    opt="adam"
    sched="step",
    lr=0.001
)

{'model_class': <class 'models.pointnetcls.PointNet'>, 'model_kwargs': {'k': 2}, 'epochs': 1, 'batch_size': 8, 'num_workers': 2, 'norm': True, 'checkpoint_epoch': None, 'model_name': 'PointNet', 'intra_root': './data', 'npoints': 1024, 'exclude_seg': True, 'snapshot_path': './snapshots', 'splits': './file_splits/cls/', 'loss_fn': <function nll_loss at 0x144a383a0>, 'aug': None, 'trans_loss': False}

F1:


/Users/jventers/miniconda3/envs/masters-env/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


KeyboardInterrupt: 